### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [37]:
import numpy as np
import pandas as pd

df = pd.read_csv(r"C:\Users\user\Downloads\data.csv")
df["mese_numerico"]=[i+1 for i in range(len(df))]
df=df[df.passengers.notna()]
df

,date,passengers,mese_numerico
0,1949-01,112.0,1
1,1949-02,118.0,2
2,1949-03,132.0,3
3,1949-04,129.0,4
4,1949-05,121.0,5
...,...,...,...
126,1960-07,622.0,127
127,1960-08,606.0,128
128,1960-09,508.0,129
129,1960-10,461.0,130


In [45]:
import matplotlib.pyplot as plt
import scipy as sp
import math
df = df.reset_index(drop=True)

def fun(x, a, b, c, d,e):
    return a * x**4 + b * x**3 + c*x**2 + d*x +e
params,_=sp.optimize.curve_fit(fun,df.mese_numerico,df.passengers)
y_fit=fun(df.mese_numerico,*params)

def calculate_rmse(y_real, y_pred):
    if len(y_real) != len(y_pred):
        raise ValueError("Le liste devono avere la stessa lunghezza.")
    mse = sum((y_real[i] - y_pred[i]) ** 2 for i in range(len(y_real))) / len(y_real)
    return math.sqrt(mse)

{"RMSE": calculate_rmse(df.passengers.values,y_fit)}

{'RMSE': 44.67863971436161}

In [48]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_scatter(x=df.mese_numerico, y=df.passengers, mode="lines", name="Dati reali")
fig.add_scatter(x=df.mese_numerico, y=y_fit, mode="lines", name="Fit polinomiale")
fig.update_layout(title="Regressione polinomiale sui passeggeri", xaxis_title="Mese numerico", yaxis_title="Numero passeggeri")
fig.show()

### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [12]:
import dash
from dash import dcc, html      
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


x=np.linspace(-3,3,100)
y = x**3 - x + np.random.normal(0, 3, size=x.shape)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("Regressione polinomiale interattiva"),
    dcc.Slider(
        id='grado-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    dcc.Graph(id='grafico-fit')
])

@app.callback(
    Output('grafico-fit', 'figure'),
    Input('grado-slider', 'value')
)
def aggiorna_grafico(grado):
    poly = PolynomialFeatures(degree=grado)
    X_poly = poly.fit_transform(x.reshape(-1, 1))
    model = LinearRegression().fit(X_poly, y)
    y_fit = model.predict(X_poly)

    fig = go.Figure()
    fig.add_scatter(x=x, y=y, mode='markers', name='Dati')
    fig.add_scatter(x=x, y=y_fit, mode='lines', name=f'Fit grado {grado}')
    fig.update_layout(title=f"Regressione polinomiale (grado {grado})",
                      xaxis_title="x", yaxis_title="y")
    return fig

if __name__ == '__main__':
    app.run(debug=True)